<a href="https://colab.research.google.com/github/ORAzzQWQ/NLP_2024/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random

In [7]:
!gdown --id 1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE- -O arithmetic_NLP.zip
!unzip arithmetic_NLP.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-
From (redirected): https://drive.google.com/uc?id=1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-&confirm=t&uuid=e3ad78d0-d11a-471d-86fb-817862aa5e88
To: /content/arithmetic_NLP.zip
100% 27.3M/27.3M [00:00<00:00, 85.2MB/s]
Archive:  arithmetic_NLP.zip
  inflating: arithmetic_eval.csv     
  inflating: arithmetic_train.csv    


In [8]:
df_train = pd.read_csv('arithmetic_train.csv')
df_eval = pd.read_csv('arithmetic_eval.csv')
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [9]:
# Transform the output data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

In [10]:
char_to_id = {}
id_to_char = {}

characters = ['<pad>', '<eos>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '*', '(', ')', '=']
for idx, char in enumerate(characters):
    char_to_id[char] = idx
    id_to_char[idx] = char

vocab_size = len(char_to_id)

print('vocab_size: {}'.format(vocab_size))

vocab_size: 18


In [11]:
def char_id(expr, token_map):
    tokens = re.findall(r'\d|[+\-*/=()]', expr)
    id_list = [token_map[token] for token in tokens if token in token_map]
    id_list.append(token_map['<eos>'])
    return id_list

def label_id(char_id_list, token_map):
    equal_pos = char_id_list.index(token_map['='])
    return [0] * (equal_pos+1) + char_id_list[equal_pos + 1:]

df_train['char_id_list'] = df_train['src'].apply(lambda x: char_id(x, char_to_id))
df_train['label_id_list'] = df_train['char_id_list'].apply(label_id, token_map=char_to_id)  # 等號後的
df_train = df_train[['src', 'tgt', 'len', 'char_id_list', 'label_id_list']]

df_eval['char_id_list'] = df_eval['src'].apply(lambda x: char_id(x, char_to_id))
df_eval['label_id_list'] = df_eval['char_id_list'].apply(label_id, token_map=char_to_id)
df_eval = df_eval[['src', 'tgt', 'len', 'char_id_list', 'label_id_list']]

df_train.head()
# df_train.to_csv('df_train.csv', index=False)
# df_eval.to_csv('df_eval.csv', index=False)

,src,tgt,len,char_id_list,label_id_list
0,14*(43+20)=882,882,14,"[3, 6, 14, 15, 6, 5, 12, 4, 2, 16, 17, 10, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 4, 1]"
1,(6+1)*5=35,35,10,"[15, 8, 12, 3, 16, 14, 7, 17, 5, 7, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 5, 7, 1]"
2,13+32+29=74,74,11,"[3, 5, 12, 5, 4, 12, 4, 11, 17, 9, 6, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 6, 1]"
3,31*(3-11)=-248,-248,14,"[5, 3, 14, 15, 5, 13, 3, 3, 16, 17, 13, 4, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 4, 6, 10, 1]"
4,24*49+1=1177,1177,12,"[4, 6, 14, 6, 11, 12, 3, 17, 3, 3, 9, 9, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 9, 9, 1]"


In [16]:
# Model (ASK Claude and TA example)
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        # return how much data is here in the Dataset object
        return len(self.sequences)

    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        data = self.sequences.iloc[index]
        x = torch.tensor(data['char_id_list'])
        y = torch.tensor(data['label_id_list'])
        return x, y

# Model Implementation
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()  # 修正：加上括號

        # Embedding layer
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=embed_dim,
                                          padding_idx=char_to_id['<pad>'])

        # Two LSTM layers
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                       hidden_size=hidden_dim,
                                       batch_first=True)

        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                       hidden_size=hidden_dim,
                                       batch_first=True)

        # Sequential layer with linear transformations and ReLU
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)
        )

    def forward(self, x, target=None):
        # x shape: (batch_size, sequence_length)
        batch_size = x.size(0)
        sequence_length = x.size(1)

        # 1. 嵌入層處理所有輸入
        embedded = self.embedding(x)
        # embedded shape: (batch_size, sequence_length, embed_dim)

        # 2. 通過 LSTM 層
        output1, _ = self.rnn_layer1(embedded)
        output2, _ = self.rnn_layer2(output1)
        # output2 shape: (batch_size, sequence_length, hidden_dim)

        # 3. 通過線性層得到預測
        outputs = self.linear(output2)
        # outputs shape: (batch_size, sequence_length, vocab_size)

        return outputs

    def generator(self, start_char, max_len=200):
        # Convert input characters to IDs
        char_list = [char_to_id[c] for c in start_char]

        next_char = None

        while len(char_list) < max_len:
            # Pack the char_list to tensor
            x = torch.tensor(char_list).unsqueeze(0).to(next(self.parameters()).device)

            # Input the tensor through the model layers
            embedded = self.embedding(x)
            output1, _ = self.rnn_layer1(embedded)
            output2, _ = self.rnn_layer2(output1)
            y = self.linear(output2)

            # Obtain the next token prediction
            y = y[:, -1, :]  # Get the last prediction

            # Use argmax function to get the next token prediction
            next_char = torch.argmax(y, dim=-1).item()

            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)

        # Convert IDs back to characters
        return [id_to_char[ch_id] for ch_id in char_list]

In [19]:
# (ASK Claude and ChatGPT)
def collate_fn(batch):
    # 把序列和標籤分開
    sequences, labels = zip(*batch)

    # 使用 pad_sequence 自動處理填充
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=char_to_id['<pad>'])
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=char_to_id['<pad>'])

    return padded_sequences, padded_labels

def train_step(model, optimizer, criterion, x, y):
    # 1. 獲取模型預測
    logits = model(x)

    # 2. 計算損失（注意：y需要錯一位，因為我們在預測下一個字符）
    loss = criterion(
        logits[:, :-1].reshape(-1, logits.size(-1)),  # 除去最後一個預測
        y[:, 1:].reshape(-1)  # 除去第一個目標
    )

    # 3. 反向傳播和優化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

# 使用示例


In [ ]:
# (ASK Claude and ChatGPT)
model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

num_epochs = 10
train_dataset = Dataset(df_train)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,  # 減小 batch_size
    shuffle=True,
    collate_fn=collate_fn
)

# 訓練循環
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    batch_count = 0

    try:
        for batch_idx, (x, y) in enumerate(train_loader):
            loss = train_step(model, optimizer, criterion, x, y)
            total_loss += loss
            batch_count += 1

            if (batch_idx + 1) % 1000 == 0:
                avg_loss = total_loss / batch_count
                print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss:.4f}, Avg Loss: {avg_loss:.4f}")

        # 每個 epoch 結束後更新學習率
        epoch_loss = total_loss / batch_count
        scheduler.step(epoch_loss)
        print(f"Epoch {epoch+1} completed, Average Loss: {epoch_loss:.4f}")

    except Exception as e:
        print(f"Error in epoch {epoch+1}: {str(e)}")
        continue

# 保存模型
torch.save(model.state_dict(), 'char_rnn_model.pth')

# # 加载模型
# model.load_state_dict(torch.load('char_rnn_model.pth'))
# model.eval()


In [25]:
# (ASK Claude and ChatGPT)
def evaluate_model(model, eval_loader, device, char_to_id, id_to_char):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

    # 用于存储每个字符的准确率统计
    char_stats = {char: {'correct': 0, 'total': 0} for char in char_to_id.keys()}

    with torch.no_grad():
        for batch_idx, (input_seq, target_seq) in enumerate(eval_loader):
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            # 获取模型输出
            output = model(input_seq)

            # 计算损失
            loss = criterion(
                output[:, :-1].reshape(-1, len(char_to_id)),  # 除去最后一个预测
                target_seq[:, 1:].reshape(-1)  # 除去第一个目标
            )
            total_loss += loss.item()

            # 获取预测
            _, predicted = torch.max(output[:, :-1].reshape(-1, len(char_to_id)), dim=1)
            targets = target_seq[:, 1:].reshape(-1)

            # 只考虑非填充字符的预测
            mask = targets != char_to_id['<pad>']
            predicted = predicted[mask]
            targets = targets[mask]

            # 统计每个字符的准确率
            for pred, targ in zip(predicted, targets):
                pred_char = id_to_char[pred.item()]
                targ_char = id_to_char[targ.item()]

                char_stats[targ_char]['total'] += 1
                if pred_char == targ_char:
                    char_stats[targ_char]['correct'] += 1
                    correct += 1
                total += 1

    accuracy = (correct / total * 100) if total > 0 else 0
    avg_loss = total_loss / len(eval_loader)

    char_accuracies = {}
    for char, stats in char_stats.items():
        if stats['total'] > 0:
            char_accuracies[char] = (stats['correct'] / stats['total'] * 100)

    return {
        'accuracy': accuracy,
        'loss': avg_loss,
        'char_accuracies': char_accuracies
    }


def main():

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    model.load_state_dict(torch.load('char_rnn_model_v3', map_location=device))
    model.to(device)
    model.eval()

    eval_dataset = Dataset(df_eval)
    eval_loader = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size=16,
        shuffle=False,
        collate_fn=collate_fn
    )

    results = evaluate_model(model, eval_loader, device, char_to_id, id_to_char)

    print(f"\nEvaluation Results:")
    print(f"Overall Accuracy: {results['accuracy']:.2f}%")
    print(f"Average Loss: {results['loss']:.4f}")
    print("\nPer-character Accuracy:")

    sorted_chars = sorted(
        results['char_accuracies'].items(),
        key=lambda x: x[1],
        reverse=True
    )

    for char, acc in sorted_chars:
        if char not in ['<pad>', '<eos>']:
            print(f"'{char}': {acc:.2f}%")

if __name__ == "__main__":
    main()

<ipython-input-25-a9ba32953fad>:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('char_rnn_model_v3', map_location=device))



Evaluation Results:
Overall Accuracy: 88.40%
Average Loss: 0.2694

Per-character Accuracy:
'-': 99.94%
'0': 93.45%
'1': 91.28%
'2': 86.78%
'5': 81.77%
'4': 80.64%
'6': 78.51%
'8': 77.92%
'9': 76.96%
'3': 76.89%
'7': 73.19%


In [ ]:
"""
1.	What impact does using different learning rates have on model training? (ASK Claude)
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import time

def prepare_subset_data(df_train, subset_ratio=0.5):
    """準備子集數據"""
    # 隨機抽樣50%的數據
    subset_size = int(len(df_train) * subset_ratio)
    subset_indices = np.random.choice(len(df_train), subset_size, replace=False)
    return df_train.iloc[subset_indices].reset_index(drop=True)

def train_with_lr(model, train_loader, learning_rate, num_epochs=5, device='cuda'):
    """使用指定學習率訓練模型"""
    model_copy = deepcopy(model)
    model_copy.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
    optimizer = optim.Adam(model_copy.parameters(), lr=learning_rate)

    history = {
        'loss': [],
        'epoch_times': []
    }

    for epoch in range(num_epochs):
        model_copy.train()
        epoch_loss = 0
        batch_count = 0
        epoch_start = time.time()

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            output = model_copy(x)

            loss = criterion(
                output[:, :-1].reshape(-1, output.size(-1)),
                y[:, 1:].reshape(-1)
            )

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            batch_count += 1

            # 每50個batch輸出一次損失
            if batch_count % 1000 == 0:
                avg_loss = epoch_loss / batch_count
                history['loss'].append(avg_loss)
                print(f"LR {learning_rate}: Epoch {epoch+1}, Batch {batch_count}, Loss: {avg_loss:.4f}")

        epoch_time = time.time() - epoch_start
        history['epoch_times'].append(epoch_time)

        print(f"\nLR {learning_rate}: Epoch {epoch+1} 完成, "
              f"平均損失: {epoch_loss/batch_count:.4f}, "
              f"耗時: {epoch_time:.2f}秒")

    return history

def analyze_learning_rates():
    """分析不同學習率的效果"""
    # 設定要測試的學習率
    learning_rates = [0.0001, 0.001, 0.01]
    histories = {}
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 準備50%的訓練數據
    subset_df = prepare_subset_data(df_train, subset_ratio=0.5)
    print(f"使用數據集大小: {len(subset_df)} (原始數據集的50%)")

    # 初始化模型和數據加載器
    base_model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    train_loader = torch.utils.data.DataLoader(
        Dataset(subset_df),
        batch_size=32,  # 稍微增加batch_size以加快訓練
        shuffle=True,
        collate_fn=collate_fn
    )

    # 使用不同學習率訓練
    for lr in learning_rates:
        print(f"\n開始訓練 - 學習率: {lr}")
        histories[lr] = train_with_lr(base_model, train_loader, lr, num_epochs=5, device=device)

    # 繪製損失曲線
    plt.figure(figsize=(12, 6))
    for lr, history in histories.items():
        plt.plot(history['loss'], label=f'學習率 = {lr}')

    plt.xlabel('訓練批次 (x50)')
    plt.ylabel('損失')
    plt.title('不同學習率的訓練損失曲線比較')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    plt.show()

    # 比較訓練時間
    plt.figure(figsize=(10, 5))
    avg_epoch_times = [np.mean(history['epoch_times']) for history in histories.values()]
    plt.bar([str(lr) for lr in learning_rates], avg_epoch_times)
    plt.xlabel('學習率')
    plt.ylabel('平均每輪訓練時間（秒）')
    plt.title('不同學習率的訓練時間比較')
    plt.show()

    # 計算和顯示結果
    print("\n學習率分析結果:")
    print("\n學習率    最終損失    平均每輪時間    損失波動性")
    print("-" * 55)
    for lr, history in histories.items():
        final_loss = history['loss'][-1]
        avg_time = np.mean(history['epoch_times'])
        loss_volatility = np.std(history['loss'])
        print(f"{lr:.4f}    {final_loss:.4f}        {avg_time:.2f}秒        {loss_volatility:.4f}")

    return histories

if __name__ == "__main__":
    results = analyze_learning_rates()

使用數據集大小: 1184625 (原始數據集的50%)

開始訓練 - 學習率: 0.0001
LR 0.0001: Epoch 1, Batch 1000, Loss: 1.9630
LR 0.0001: Epoch 1, Batch 2000, Loss: 1.8099
LR 0.0001: Epoch 1, Batch 3000, Loss: 1.7447
LR 0.0001: Epoch 1, Batch 4000, Loss: 1.7018
LR 0.0001: Epoch 1, Batch 5000, Loss: 1.6695
LR 0.0001: Epoch 1, Batch 6000, Loss: 1.6437
LR 0.0001: Epoch 1, Batch 7000, Loss: 1.6223
LR 0.0001: Epoch 1, Batch 8000, Loss: 1.6041
LR 0.0001: Epoch 1, Batch 9000, Loss: 1.5874
LR 0.0001: Epoch 1, Batch 10000, Loss: 1.5720
LR 0.0001: Epoch 1, Batch 11000, Loss: 1.5580
LR 0.0001: Epoch 1, Batch 12000, Loss: 1.5444
LR 0.0001: Epoch 1, Batch 13000, Loss: 1.5311
LR 0.0001: Epoch 1, Batch 14000, Loss: 1.5181
LR 0.0001: Epoch 1, Batch 15000, Loss: 1.5054
LR 0.0001: Epoch 1, Batch 16000, Loss: 1.4926
LR 0.0001: Epoch 1, Batch 17000, Loss: 1.4801
LR 0.0001: Epoch 1, Batch 18000, Loss: 1.4676
LR 0.0001: Epoch 1, Batch 19000, Loss: 1.4554
LR 0.0001: Epoch 1, Batch 20000, Loss: 1.4435
LR 0.0001: Epoch 1, Batch 21000, Loss: 1

In [ ]:
"""
2.If you use RNN or GRU instead of LSTM, what will happen to the quality of your answer generation? (ASK Claude)
"""

import torch
import torch.nn as nn
import numpy as np
import time
from copy import deepcopy

class BaseRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, rnn_type='lstm'):
        super(BaseRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=char_to_id['<pad>'])

        # 根據指定類型選擇RNN層
        if rnn_type == 'rnn':
            self.rnn1 = nn.RNN(embed_dim, hidden_dim, batch_first=True)
            self.rnn2 = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        elif rnn_type == 'gru':
            self.rnn1 = nn.GRU(embed_dim, hidden_dim, batch_first=True)
            self.rnn2 = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        else:  # lstm
            self.rnn1 = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
            self.rnn2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)

        self.linear = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

        self.rnn_type = rnn_type

    def forward(self, x):
        embedded = self.embedding(x)
        output1, _ = self.rnn1(embedded)
        output2, _ = self.rnn2(output1)
        return self.linear(output2)

def train_and_evaluate(model_type, train_loader, eval_loader, device, epochs=5):
    """訓練並評估指定類型的模型"""
    model = BaseRNN(
        vocab_size=len(char_to_id),
        embed_dim=64,
        hidden_dim=128,
        rnn_type=model_type
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    history = {
        'train_loss': [],
        'eval_loss': [],
        'eval_accuracy': [],
        'epoch_times': []
    }

    for epoch in range(epochs):
        # 訓練階段
        model.train()
        epoch_start = time.time()
        train_loss = 0
        batch_count = 0

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            output = model(x)
            loss = criterion(
                output[:, :-1].reshape(-1, output.size(-1)),
                y[:, 1:].reshape(-1)
            )

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            batch_count += 1

            if batch_count % 50 == 0:
                print(f"{model_type.upper()} - Epoch {epoch+1}, Batch {batch_count}, Loss: {loss.item():.4f}")

        avg_train_loss = train_loss / batch_count
        epoch_time = time.time() - epoch_start

        # 評估階段
        model.eval()
        eval_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for x, y in eval_loader:
                x, y = x.to(device), y.to(device)
                output = model(x)

                # 計算損失
                loss = criterion(
                    output[:, :-1].reshape(-1, output.size(-1)),
                    y[:, 1:].reshape(-1)
                )
                eval_loss += loss.item()

                # 計算準確率
                _, predicted = torch.max(output[:, :-1].reshape(-1, output.size(-1)), 1)
                targets = y[:, 1:].reshape(-1)
                mask = targets != char_to_id['<pad>']
                correct += (predicted[mask] == targets[mask]).sum().item()
                total += mask.sum().item()

        avg_eval_loss = eval_loss / len(eval_loader)
        accuracy = (correct / total * 100) if total > 0 else 0

        # 保存結果
        history['train_loss'].append(avg_train_loss)
        history['eval_loss'].append(avg_eval_loss)
        history['eval_accuracy'].append(accuracy)
        history['epoch_times'].append(epoch_time)

        print(f"\n{model_type.upper()} Epoch {epoch+1} 結果:")
        print(f"訓練損失: {avg_train_loss:.4f}")
        print(f"評估損失: {avg_eval_loss:.4f}")
        print(f"準確率: {accuracy:.2f}%")
        print(f"耗時: {epoch_time:.2f}秒\n")

    return history

def compare_models():
    """比較不同RNN模型的性能"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 準備數據
    subset_df = df_train.sample(frac=0.5, random_state=42)
    train_loader = torch.utils.data.DataLoader(
        Dataset(subset_df),
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )

    eval_loader = torch.utils.data.DataLoader(
        Dataset(df_eval),
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )

    # 訓練並評估每種模型
    model_types = ['rnn', 'gru', 'lstm']
    results = {}

    for model_type in model_types:
        print(f"\n開始訓練 {model_type.upper()} 模型...")
        results[model_type] = train_and_evaluate(
            model_type, train_loader, eval_loader, device
        )

    return results

if __name__ == "__main__":
    results = compare_models()

In [ ]:
"""
3.  If construct an evaluation set using three-digit numbers while the training set is constructed from two-digit numbers,
    what will happen to the quality of your answer gener we ation? (ASK Claude)
"""

import torch
import pandas as pd
import random
import re

def generate_three_digit_eval_data(num_samples=1000):
    """生成三位數的算術驗證數據"""
    data = []
    operations = ['+', '-', '*']

    for _ in range(num_samples):
        # 生成三位數
        num1 = random.randint(100, 999)
        num2 = random.randint(100, 999)
        op = random.choice(operations)

        # 計算結果
        if op == '+':
            result = num1 + num2
        elif op == '-':
            # 確保結果為正數
            if num1 < num2:
                num1, num2 = num2, num1
            result = num1 - num2
        else:  # '*'
            result = num1 * num2

        expr = f"{num1}{op}{num2}"

        data.append({
            'src': expr,
            'tgt': str(result)
        })

    df = pd.DataFrame(data)

    # 添加與原始數據集相同的處理
    df['tgt'] = df['tgt'].apply(lambda x: str(x))
    df['src'] = df['src'].add(df['tgt'])
    df['len'] = df['src'].apply(lambda x: len(x))

    # 添加字符ID列表
    df['char_id_list'] = df['src'].apply(lambda x: char_id(x, char_to_id))
    df['label_id_list'] = df['char_id_list'].apply(label_id, token_map=char_to_id)

    return df[['src', 'tgt', 'len', 'char_id_list', 'label_id_list']]

def evaluate_model(model, eval_loader, device):
    """評估模型性能"""
    model.eval()
    total_correct = 0
    total_samples = 0
    operation_stats = {'+': {'correct': 0, 'total': 0},
                      '-': {'correct': 0, 'total': 0},
                      '*': {'correct': 0, 'total': 0}}
    error_examples = []

    criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
    total_loss = 0

    with torch.no_grad():
        for batch_idx, (input_seq, target_seq) in enumerate(eval_loader):
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            # 獲取模型輸出
            output = model(input_seq)

            # 計算損失
            loss = criterion(
                output[:, :-1].reshape(-1, len(char_to_id)),
                target_seq[:, 1:].reshape(-1)
            )
            total_loss += loss.item()

            # 對每個樣本進行評估
            for i in range(input_seq.size(0)):
                # 解碼輸入序列（原始算式）
                input_chars = [id_to_char[id.item()] for id in input_seq[i]
                             if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                input_expr = ''.join(input_chars)

                # 提取運算符
                op = next(c for c in input_expr if c in ['+', '-', '*'])

                # 解碼目標序列（正確答案）
                target_chars = [id_to_char[id.item()] for id in target_seq[i]
                              if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                target_ans = ''.join(target_chars[target_chars.index('=')+1:])  # 只取等號後的答案部分

                # 獲取模型預測
                pred_logits = output[i]
                pred_ids = torch.argmax(pred_logits, dim=-1)
                pred_chars = [id_to_char[id.item()] for id in pred_ids
                            if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                pred_ans = ''.join(pred_chars[pred_chars.index('=')+1:])  # 只取等號後的答案部分

                # 更新統計信息
                operation_stats[op]['total'] += 1
                if pred_ans == target_ans:
                    total_correct += 1
                    operation_stats[op]['correct'] += 1
                else:
                    # 記錄錯誤示例
                    error_examples.append({
                        'expression': input_expr,
                        'correct': target_ans,
                        'predicted': pred_ans,
                        'operation': op
                    })

                total_samples += 1

    # 計算準確率
    accuracy = total_correct / total_samples * 100
    avg_loss = total_loss / len(eval_loader)

    # 計算每種運算的準確率
    operation_accuracy = {
        op: (stats['correct'] / stats['total'] * 100) if stats['total'] > 0 else 0
        for op, stats in operation_stats.items()
    }

    return {
        'accuracy': accuracy,
        'loss': avg_loss,
        'operation_accuracy': operation_accuracy,
        'error_examples': error_examples[:10],  # 只返回前10個錯誤示例
        'total_samples': total_samples
    }

def compare_performance():
    """比較模型在二位數和三位數上的表現"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加載模型
    model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    model.load_state_dict(torch.load('char_rnn_model_v3', map_location=device))
    model.to(device)

    # 準備三位數評估數據
    df_eval_3digit = generate_three_digit_eval_data(num_samples=1000)
    eval_loader_3digit = torch.utils.data.DataLoader(
        Dataset(df_eval_3digit),
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )

    # 評估原始二位數和新的三位數數據集
    eval_loader_2digit = torch.utils.data.DataLoader(
        Dataset(df_eval),
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )

    print("開始評估...")
    results_2digit = evaluate_model(model, eval_loader_2digit, device)
    results_3digit = evaluate_model(model, eval_loader_3digit, device)

    # 顯示結果
    print("\n=== 評估結果比較 ===")
    print(f"\n二位數測試集:")
    print(f"整體準確率: {results_2digit['accuracy']:.2f}%")
    print(f"平均損失: {results_2digit['loss']:.4f}")
    print("\n各運算準確率:")
    for op, acc in results_2digit['operation_accuracy'].items():
        print(f"{op}: {acc:.2f}%")

    print(f"\n三位數測試集:")
    print(f"整體準確率: {results_3digit['accuracy']:.2f}%")
    print(f"平均損失: {results_3digit['loss']:.4f}")
    print("\n各運算準確率:")
    for op, acc in results_3digit['operation_accuracy'].items():
        print(f"{op}: {acc:.2f}%")

    print("\n三位數錯誤示例:")
    for i, error in enumerate(results_3digit['error_examples'], 1):
        print(f"\n錯誤示例 {i}:")
        print(f"算式: {error['expression']}")
        print(f"正確答案: {error['correct']}")
        print(f"模型預測: {error['predicted']}")
        print(f"運算類型: {error['operation']}")

    return results_2digit, results_3digit

if __name__ == "__main__":
    results_2digit, results_3digit = compare_performance()

In [ ]:
"""
4.	If some numbers never appear in your training data, what will happen to your answer generation? (ASK Claude)
"""


import torch
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

def generate_data_with_missing_numbers(num_samples=1000, missing_numbers=[7]):
    """生成不包含特定數字的訓練數據"""
    data = []
    operations = ['+', '-', '*']

    def number_contains_digit(n, digits):
        """檢查數字是否包含特定數字"""
        return any(str(d) in str(n) for d in digits)

    def generate_valid_number(min_val, max_val, missing_digits):
        """生成不包含特定數字的隨機數"""
        while True:
            num = random.randint(min_val, max_val)
            if not number_contains_digit(num, missing_digits):
                return num

    # 生成訓練數據
    for _ in range(num_samples):
        num1 = generate_valid_number(10, 99, missing_numbers)
        num2 = generate_valid_number(10, 99, missing_numbers)
        op = random.choice(operations)

        # 計算結果
        if op == '+':
            result = num1 + num2
        elif op == '-':
            if num1 < num2:
                num1, num2 = num2, num1
            result = num1 - num2
        else:  # '*'
            result = num1 * num2

        data.append({
            'src': f"{num1}{op}{num2}",
            'tgt': str(result)
        })

    df = pd.DataFrame(data)
    df['tgt'] = df['tgt'].apply(str)
    df['src'] = df['src'].add(df['tgt'])
    df['len'] = df['src'].apply(len)

    return df

def generate_test_data_with_missing_numbers(num_samples=200):
    """生成包含被排除數字的測試數據"""
    data = []
    operations = ['+', '-', '*']

    for _ in range(num_samples):
        # 確保至少一個數字包含7
        if random.random() < 0.5:
            num1 = random.randint(70, 79)  # 確保包含7
            num2 = random.randint(10, 99)
        else:
            num1 = random.randint(10, 99)
            num2 = random.randint(70, 79)  # 確保包含7

        op = random.choice(operations)

        # 計算結果
        if op == '+':
            result = num1 + num2
        elif op == '-':
            if num1 < num2:
                num1, num2 = num2, num1
            result = num1 - num2
        else:  # '*'
            result = num1 * num2

        data.append({
            'src': f"{num1}{op}{num2}",
            'tgt': str(result)
        })

    df = pd.DataFrame(data)
    df['tgt'] = df['tgt'].apply(str)
    df['src'] = df['src'].add(df['tgt'])
    df['len'] = df['src'].apply(len)

    return df

def analyze_errors(true_vals, pred_vals):
    """分析預測錯誤的模式"""
    error_patterns = []

    for true, pred in zip(true_vals, pred_vals):
        if true != pred:
            error_patterns.append({
                'true': true,
                'predicted': pred,
                'true_contains_7': '7' in true,
                'error_type': 'substitution' if len(true) == len(pred) else 'length_mismatch'
            })

    return error_patterns

def evaluate_model_on_missing_numbers(model, test_loader, device):
    """評估模型在包含缺失數字的數據上的表現"""
    model.eval()
    results = {
        'correct': 0,
        'total': 0,
        'errors': [],
        'predictions': [],
        'targets': []
    }

    with torch.no_grad():
        for batch_idx, (input_seq, target_seq) in enumerate(test_loader):
            input_seq = input_seq.to(device)
            target_seq = target_seq.to(device)

            # 獲取模型輸出
            output = model(input_seq)

            # 解碼預測和目標
            for i in range(input_seq.size(0)):
                # 解碼輸入
                input_chars = [id_to_char[id.item()] for id in input_seq[i]
                             if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                input_expr = ''.join(input_chars)

                # 解碼目標
                target_chars = [id_to_char[id.item()] for id in target_seq[i]
                              if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                target_ans = ''.join(target_chars[target_chars.index('=')+1:])

                # 解碼預測
                pred_logits = output[i]
                pred_ids = torch.argmax(pred_logits, dim=-1)
                pred_chars = [id_to_char[id.item()] for id in pred_ids
                            if id.item() != char_to_id['<pad>'] and id.item() != char_to_id['<eos>']]
                pred_ans = ''.join(pred_chars[pred_chars.index('=')+1:])

                # 記錄結果
                results['total'] += 1
                if pred_ans == target_ans:
                    results['correct'] += 1
                else:
                    results['errors'].append({
                        'expression': input_expr,
                        'true': target_ans,
                        'predicted': pred_ans,
                        'contains_7': '7' in input_expr
                    })

                results['predictions'].append(pred_ans)
                results['targets'].append(target_ans)

    return results

def main():
    # 設置設備
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 生成不含7的訓練數據
    train_data = generate_data_with_missing_numbers(num_samples=1000, missing_numbers=[7])
    test_data = generate_test_data_with_missing_numbers(num_samples=200)

    # 準備數據加載器
    train_dataset = Dataset(train_data)
    test_dataset = Dataset(test_data)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        collate_fn=collate_fn
    )

    # 加載模型
    model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    model.load_state_dict(torch.load('char_rnn_model_v3', map_location=device))
    model.to(device)

    # 評估模型
    results = evaluate_model_on_missing_numbers(model, test_loader, device)

    # 分析結果
    accuracy = results['correct'] / results['total'] * 100
    error_patterns = analyze_errors(results['targets'], results['predictions'])

    print(f"\n=== 評估結果 ===")
    print(f"整體準確率: {accuracy:.2f}%")
    print(f"總樣本數: {results['total']}")
    print(f"正確預測數: {results['correct']}")

    print("\n=== 錯誤分析 ===")
    error_types = Counter([e['error_type'] for e in error_patterns])
    print("\n錯誤類型分布:")
    for error_type, count in error_types.items():
        print(f"{error_type}: {count}")

    print("\n包含7的錯誤示例:")
    for error in [e for e in results['errors'] if e['contains_7']][:5]:
        print(f"\n算式: {error['expression']}")
        print(f"正確答案: {error['true']}")
        print(f"模型預測: {error['predicted']}")

    return results

if __name__ == "__main__":
    results = main()

In [ ]:
"""
5.	Why do we need gradient clipping during training? (ASK Claude)
"""

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import time

def train_with_clipping(model, train_loader, clip_value, num_epochs=5):
    """使用梯度裁剪的訓練函數"""
    device = next(model.parameters()).device
    model_copy = deepcopy(model)
    optimizer = torch.optim.Adam(model_copy.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

    history = {
        'loss': [],
        'gradients': [],
        'clipped_gradients': []
    }

    for epoch in range(num_epochs):
        model_copy.train()
        epoch_loss = 0
        batch_count = 0

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # 前向傳播
            output = model_copy(x)
            loss = criterion(
                output[:, :-1].reshape(-1, output.size(-1)),
                y[:, 1:].reshape(-1)
            )

            # 反向傳播
            loss.backward()

            # 記錄原始梯度
            original_grad_norm = torch.nn.utils.clip_grad_norm_(
                model_copy.parameters(), float('inf')
            )
            history['gradients'].append(original_grad_norm.item())

            # 應用梯度裁剪
            clipped_grad_norm = torch.nn.utils.clip_grad_norm_(
                model_copy.parameters(), clip_value
            )
            history['clipped_gradients'].append(clipped_grad_norm.item())

            optimizer.step()

            epoch_loss += loss.item()
            batch_count += 1

            if batch_count % 50 == 0:
                avg_loss = epoch_loss / batch_count
                history['loss'].append(avg_loss)
                print(f"Epoch {epoch+1}, Batch {batch_count}, "
                      f"Loss: {avg_loss:.4f}, "
                      f"Gradient Norm: {original_grad_norm:.4f}, "
                      f"Clipped Norm: {clipped_grad_norm:.4f}")

    return history

def analyze_gradient_clipping():
    """分析不同梯度裁剪閾值的效果"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 準備數據
    train_loader = torch.utils.data.DataLoader(
        Dataset(df_train),
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )

    # 測試不同的裁剪閾值
    clip_values = [1.0, 5.0, 10.0]
    results = {}

    base_model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    base_model.to(device)

    for clip_value in clip_values:
        print(f"\n訓練模型 - 梯度裁剪閾值: {clip_value}")
        results[clip_value] = train_with_clipping(
            base_model, train_loader, clip_value
        )

    # 繪製結果
    plt.figure(figsize=(15, 5))

    # 損失曲線
    plt.subplot(1, 2, 1)
    for clip_value, history in results.items():
        plt.plot(history['loss'], label=f'Clip={clip_value}')
    plt.xlabel('Batch (x50)')
    plt.ylabel('Loss')
    plt.title('Training Loss with Different Gradient Clipping')
    plt.legend()
    plt.grid(True)

    # 梯度分布
    plt.subplot(1, 2, 2)
    for clip_value, history in results.items():
        plt.hist(history['gradients'],
                bins=50,
                alpha=0.5,
                label=f'Clip={clip_value}',
                density=True)
    plt.xlabel('Gradient Norm')
    plt.ylabel('Density')
    plt.title('Gradient Norm Distribution')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return results

if __name__ == "__main__":
    results = analyze_gradient_clipping()

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import time

def train_with_clipping(model, train_loader, clip_value, num_epochs=5):
    """使用梯度裁剪的訓練函數"""
    device = next(model.parameters()).device
    model_copy = deepcopy(model)
    optimizer = torch.optim.Adam(model_copy.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])

    history = {
        'loss': [],
        'gradients': [],
        'clipped_gradients': []
    }

    for epoch in range(num_epochs):
        model_copy.train()
        epoch_loss = 0
        batch_count = 0

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # 前向傳播
            output = model_copy(x)
            loss = criterion(
                output[:, :-1].reshape(-1, output.size(-1)),
                y[:, 1:].reshape(-1)
            )

            # 反向傳播
            loss.backward()

            # 記錄原始梯度
            original_grad_norm = torch.nn.utils.clip_grad_norm_(
                model_copy.parameters(), float('inf')
            )
            history['gradients'].append(original_grad_norm.item())

            # 應用梯度裁剪
            clipped_grad_norm = torch.nn.utils.clip_grad_norm_(
                model_copy.parameters(), clip_value
            )
            history['clipped_gradients'].append(clipped_grad_norm.item())

            optimizer.step()

            epoch_loss += loss.item()
            batch_count += 1

            if batch_count % 50 == 0:
                avg_loss = epoch_loss / batch_count
                history['loss'].append(avg_loss)
                print(f"Epoch {epoch+1}, Batch {batch_count}, "
                      f"Loss: {avg_loss:.4f}, "
                      f"Gradient Norm: {original_grad_norm:.4f}, "
                      f"Clipped Norm: {clipped_grad_norm:.4f}")

    return history

def analyze_gradient_clipping():
    """分析不同梯度裁剪閾值的效果"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 準備數據
    train_loader = torch.utils.data.DataLoader(
        Dataset(df_train),
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )

    # 測試不同的裁剪閾值
    clip_values = [1.0, 5.0, 10.0]
    results = {}

    base_model = CharRNN(vocab_size=len(char_to_id), embed_dim=64, hidden_dim=128)
    base_model.to(device)

    for clip_value in clip_values:
        print(f"\n訓練模型 - 梯度裁剪閾值: {clip_value}")
        results[clip_value] = train_with_clipping(
            base_model, train_loader, clip_value
        )

    # 繪製結果
    plt.figure(figsize=(15, 5))

    # 損失曲線
    plt.subplot(1, 2, 1)
    for clip_value, history in results.items():
        plt.plot(history['loss'], label=f'Clip={clip_value}')
    plt.xlabel('Batch (x50)')
    plt.ylabel('Loss')
    plt.title('Training Loss with Different Gradient Clipping')
    plt.legend()
    plt.grid(True)

    # 梯度分布
    plt.subplot(1, 2, 2)
    for clip_value, history in results.items():
        plt.hist(history['gradients'],
                bins=50,
                alpha=0.5,
                label=f'Clip={clip_value}',
                density=True)
    plt.xlabel('Gradient Norm')
    plt.ylabel('Density')
    plt.title('Gradient Norm Distribution')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return results

if __name__ == "__main__":
    results = analyze_gradient_clipping()